In [1]:
import numpy as np 
import pandas as pd 

import random as rn

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.stats import uniform

import itertools

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from keras.utils.np_utils import to_categorical
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import AvgPool2D, BatchNormalization, Reshape
from keras.optimizers import Adadelta, RMSprop, Adam
from keras.losses import categorical_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

import os


E:\py\envs\tf-keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv')
y= data.iloc[:,0] 

data= data.drop("label", axis=1)

x= data

#standardize my data
sc = StandardScaler()
X = sc.fit_transform(x)

#split into training and tessting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [3]:
#adjust data
import scipy.misc as smp


for x in X_train:
    for n, i in enumerate(x):
        if i > 0:
            x[n] = 0
        else:
            x[n] = 1

for x in X_test:
    for n, i in enumerate(x):
        if i > 0:
            x[n] = 0
        else:
            x[n] = 1
# B= np.reshape(X_train[90], (-1, 28))
# data = B

# img = smp.toimage( data )       # Create a PIL image
# img.show()

In [4]:
# #adjust data
# import scipy.misc as smp


# for x in data.values:
#     for n, i in enumerate(x):
#         if i > 0:
#             x[n] = 0
#         else:
#             x[n] = 1
# # B= np.reshape(X_train[90], (-1, 28))
# # data = B

# # img = smp.toimage( data )       # Create a PIL image
# # img.show()

In [5]:
#reshape
X_train = X_train.reshape(-1,28,28,1)

In [6]:
# Tensorflow Keras CNN Model
# model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Conv2D(32, (3,3), padding = "same", activation = "relu", input_shape = X_train.shape[1:]))
# model.add(tf.keras.layers.MaxPool2D(2,2))

# model.add(tf.keras.layers.Conv2D(64, (3,3), padding = "same", activation = "relu"))
# model.add(tf.keras.layers.MaxPool2D(2,2))

# model.add(tf.keras.layers.Conv2D(128, (3,3), padding = "same", activation = "relu"))
# model.add(tf.keras.layers.MaxPool2D(2,2))

# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))


# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = X_train.shape[1:]))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [7]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy']) # loss = "categorical_crossentropy",

model.summary()


# Compile the model
# model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        51232     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [8]:
model.fit(X_train, y_train, epochs=27, batch_size=86)


Epoch 1/27
29400/29400 [==============================] - 319s 11ms/step - loss: 0.3690 - acc: 0.8773
Epoch 2/27
29400/29400 [==============================] - 322s 11ms/step - loss: 0.1002 - acc: 0.9694
Epoch 3/27
29400/29400 [==============================] - 334s 11ms/step - loss: 0.0758 - acc: 0.9777
Epoch 4/27
29400/29400 [==============================] - 347s 12ms/step - loss: 0.0565 - acc: 0.9826
Epoch 5/27
29400/29400 [==============================] - 389s 13ms/step - loss: 0.0496 - acc: 0.9835
Epoch 6/27
29400/29400 [==============================] - 381s 13ms/step - loss: 0.0422 - acc: 0.9864
Epoch 7/27
29400/29400 [==============================] - 381s 13ms/step - loss: 0.0394 - acc: 0.9882
Epoch 8/27
29400/29400 [==============================] - 362s 12ms/step - loss: 0.0374 - acc: 0.9879
Epoch 9/27
29400/29400 [==============================] - 344s 12ms/step - loss: 0.0347 - acc: 0.9895
Epoch 10/27
29400/29400 [==============================] - 349s 12ms/step - loss: 

In [9]:
X_test = X_test.reshape(-1,28,28,1)
val_loss, val_acc = model.evaluate(X_test, y_test) 

val_acc

12600/12600 [==============================] - 54s 4ms/step


0.9896031746031746

In [10]:
# data = pd.concat([y, data], axis=1)
# data.head(5)

In [11]:
# from sklearn import ensemble, gaussian_process, linear_model, naive_bayes, svm, neighbors, tree, discriminant_analysis, model_selection

# MLA = [
#   #  Ensemble Methods
#     ensemble.AdaBoostClassifier(),
#     ensemble.BaggingClassifier(),
#     ensemble.ExtraTreesClassifier(),
#     ensemble.GradientBoostingClassifier(),
#     ensemble.RandomForestClassifier(),

#   #  Gaussian Processes
#     gaussian_process.GaussianProcessClassifier(),
    
#     #GLM
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(),
#     linear_model.Perceptron(),
    
#     #Navies Bayes
#     naive_bayes.BernoulliNB(),
#     naive_bayes.GaussianNB(),
    
#     #Nearest Neighbor
#     neighbors.KNeighborsClassifier(),
#         #SVM
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
#     svm.LinearSVC(),
    
#     #Trees    
#     tree.DecisionTreeClassifier(),
#     tree.ExtraTreeClassifier(),
    
#     #Discriminant Analysis
#     discriminant_analysis.LinearDiscriminantAnalysis(),
#     discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
# #     #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
# #     XGBClassifier()    
#     ]



# #split dataset in cross-validation with this splitter class:
# cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .33, train_size = .66, random_state = 0 ) 

# #create table to compare MLA metrics
# MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
# MLA_compare = pd.DataFrame(columns = MLA_columns)

# #create table to compare MLA predictions
# MLA_predict = data.iloc[:,0]

# #index through MLA and save performance to table
# row_index = 0
# for alg in MLA:

#     #set name and parameters
#     MLA_name = alg.__class__.__name__
#     MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
#     MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
#     #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
#     cv_results = model_selection.cross_validate(alg, data.iloc[:,1:], data.iloc[:,0], cv  = cv_split)

#     MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
#     MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
#     MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
#     #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
#     MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

#     #save MLA predictions - see section 6 for usage
#     alg.fit(data.iloc[:,1:], data.iloc[:,0])
#     MLA_predict[MLA_name] = alg.predict(data.iloc[:,1:])
#     row_index+=1

    
# #print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
# MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
# MLA_compare
# #MLA_predict

In [12]:
# #print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
# MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
# MLA_compare
# #MLA_predict

In [13]:
test = pd.read_csv('test.csv')
test.head()

x= test

#standardize my data
sc = StandardScaler()
test = sc.fit_transform(x)

for x in test:
    for n, i in enumerate(x):
        if i > 0:
            x[n] = 0
        else:
            x[n] = 1

#reshape
test = test.reshape(-1,28,28,1)

test_pred = model.predict(test)

submission = pd.DataFrame()
submission['ImageId'] = range(1, (len(test)+1))
submission['Label'] = np.argmax(test_pred, axis=1)

submission.head()


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [14]:
submission.to_csv("submission.csv", index=False)